In [9]:
import pandas as pd 
import numpy as np

Sentiment_data = pd.read_excel("Sentiment_Preprocessing.xlsx", usecols=["Label", "Deskripsi_tokens_stemmed"])
Sentiment_data.columns = ["Deskripsi", "Label"]

Sentiment_data.head()

,Deskripsi,Label
0,[],1
1,[],1
2,"['struggle', 'lancar']",1
3,"['aman', 'kendali']",1
4,"['terjun', 'langsung', 'rasa', 'karyawan', 'al...",1


In [10]:
import ast

def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
Sentiment_data["Sentiment_join"] = Sentiment_data["Deskripsi"].apply(join_text_list)

Sentiment_data["Sentiment_join"].head()

0                                                 
1                                                 
2                                  struggle lancar
3                                     aman kendali
4    terjun langsung rasa karyawan alam ajar dapat
Name: Sentiment_join, dtype: object

In [11]:
label = Sentiment_data["Label"]
text = Sentiment_data["Sentiment_join"]

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import joblib
# Split data menjadi train set dan test set
train_data, test_data, train_labels, test_labels = train_test_split(text, label, test_size=0.2, random_state=42)


cvect = CountVectorizer()
tfidf_mat_train = cvect.fit_transform(train_data)
#fit transform hanya pada data latih
tfidf_mat_test = cvect.transform(test_data)

joblib.dump(cvect, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [13]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Kernel yang akan digunakan
kernel_linear = 'linear'

# Inisialisasi model SVM dengan kernel linear
svm_linear = SVC(kernel=kernel_linear)

svm_linear.fit(tfidf_mat_train, train_labels)

#klasifikasi
predictions_linear = svm_linear.predict(tfidf_mat_test)

# Menghitung akurasi, precision, recall, dan F1-score dengan kernel linear
accuracy_linear = accuracy_score(test_labels, predictions_linear)
precision_linear = precision_score(test_labels, predictions_linear, average='weighted')
recall_linear = recall_score(test_labels, predictions_linear, average='weighted')
f1_linear = f1_score(test_labels, predictions_linear, average='weighted')

print("Linear Kernel:")
print("Accuracy:", accuracy_linear)
print("Precision:", precision_linear)
print("Recall:", recall_linear)
print("F1-score:", f1_linear)

cm_linear = confusion_matrix(test_labels, predictions_linear)
print("Confusion Matrix (Linear Kernel):")
print(cm_linear)


Linear Kernel:
Accuracy: 0.7333333333333333
Precision: 0.7333333333333333
Recall: 0.7333333333333333
F1-score: 0.7333333333333333
Confusion Matrix (Linear Kernel):
[[ 9  6]
 [ 6 24]]


In [14]:
import joblib

joblib.dump(svm_linear, 'sentimen.pkl')


['sentimen.pkl']

In [19]:
import joblib

# Memuat model dan TfidfVectorizer
model = joblib.load('sentimen.pkl')
tfidf = joblib.load('tfidf_vectorizer.pkl')

# Kalimat yang ingin diuji
kalimat1 = ["kadang ada yang susah dihubungi"]

# Transformasi kalimat menggunakan TfidfVectorizer yang sudah dilatih
tfidf_mat_test = tfidf.transform(kalimat1)

# Prediksi sentimen untuk kalimat pertama
hasil1 = model.predict(tfidf_mat_test)[0]
print(f"Hasil prediksi untuk kalimat pertama: {hasil1}")

Hasil prediksi untuk kalimat pertama: 0


In [20]:
import pandas as pd
import joblib

# Muat model dan TfidfVectorizer
model = joblib.load('sentimen.pkl')
tfidf = joblib.load('tfidf_vectorizer.pkl')

file_path = 'data_new 3.xlsx'
df = pd.read_excel(file_path)

# Memastikan bahwa kolom 'sentimen' ada
if 'Deskripsi' not in df.columns:
    raise ValueError("Kolom 'Deskripsi' tidak ditemukan dalam file Excel.")

# Transformasi kalimat menggunakan TfidfVectorizer
tfidf_mat_test = tfidf.transform(df['Deskripsi'])

# Prediksi sentimen dengan model
predictions = model.predict(tfidf_mat_test)

# tulis hasil prediksi di kolom baru
df['prediksi'] = predictions

# simpan hasil
output_file_path = 'hasil_prediksi_sentimen.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Hasil prediksi telah disimpan di {output_file_path}")


Hasil prediksi telah disimpan di hasil_prediksi_sentimen.xlsx
